In [1]:
import torch
import matplotlib.pyplot as plt
import random

#### callbacks

#### callbacks as GUI events

In [2]:
import ipywidgets as widgets

In [7]:
w = widgets.Button(description='Click Me!')

In [8]:
w

Button(description='Click Me!', style=ButtonStyle())

heloo...
heloo...


In [9]:
def f(o):
    print('heloo...')

In [10]:
w.on_click(f)

#### creating own callback

In [11]:
from time import sleep

In [18]:
def slow_calc(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb:
            cb(i)
    return res

In [19]:
slow_calc()

30

In [25]:
def show_progress(exl, epoch):
    print(f'{exl} we have finished epoch {epoch} so far!')

In [22]:
slow_calc(show_progress)

we have finished epoch 0 so far!
we have finished epoch 1 so far!
we have finished epoch 2 so far!
we have finished epoch 3 so far!
we have finished epoch 4 so far!


30

#### lambdas and partials

In [26]:
slow_calc(lambda x: show_progress('super budyyy',x))

super budyyy we have finished epoch 0 so far!
super budyyy we have finished epoch 1 so far!
super budyyy we have finished epoch 2 so far!
super budyyy we have finished epoch 3 so far!
super budyyy we have finished epoch 4 so far!


30

In [27]:
def show_progress(exl):
    def _inner(epoch):
        print(f'{exl} we have finished epoch {epoch} so far!')
    return _inner

In [28]:
slow_calc(show_progress('awesome budyyy'))

awesome budyyy we have finished epoch 0 so far!
awesome budyyy we have finished epoch 1 so far!
awesome budyyy we have finished epoch 2 so far!
awesome budyyy we have finished epoch 3 so far!
awesome budyyy we have finished epoch 4 so far!


30

In [29]:
def show_progress(exl, epoch):
    print(f'{exl} we have finished epoch {epoch} so far!')

In [30]:
from functools import partial

In [31]:
slow_calc(partial(show_progress, 'cooool!!!'))

cooool!!! we have finished epoch 0 so far!
cooool!!! we have finished epoch 1 so far!
cooool!!! we have finished epoch 2 so far!
cooool!!! we have finished epoch 3 so far!
cooool!!! we have finished epoch 4 so far!


30

#### callbacks as callable classes

In [34]:
class ShowProgressClass():
    def __init__(self, exl):
        self.exl = exl
    
    def __call__(self, epoch):
        print(f'{self.exl} we have finished epoch {epoch} so far!')        

In [36]:
cb = ShowProgressClass('oooohey')

In [37]:
slow_calc(cb)

oooohey we have finished epoch 0 so far!
oooohey we have finished epoch 1 so far!
oooohey we have finished epoch 2 so far!
oooohey we have finished epoch 3 so far!
oooohey we have finished epoch 4 so far!


30

#### multiple callback functions; *args and **kwargs

In [40]:
def f(*a, **kwargs):
    print(f'args {a}, kwargs {kwargs}')

In [44]:
f('olll', 'sfgsdg', size=10, weight=50)

args ('olll', 'sfgsdg'), kwargs {'size': 10, 'weight': 50}


In [45]:
def slow_calc(cb=None):
    res = 0
    for i in range(5):
        if cb:
            cb.befor_cal(i)
        res += i*i
        sleep(1)
        if cb:
            cb.after_cal(i, val=res)
    return res

In [52]:
class PrintStepCallback:
    def __init__(self):
        pass
    
    def befor_cal(self, *args, **kwargs):
        print('before the calculation')
    
    def after_cal(self, *agrs, **kwargs):
        print('after the calculation')

In [53]:
cb = PrintStepCallback()

In [54]:
slow_calc(cb)

before the calculation
after the calculation
before the calculation
after the calculation
before the calculation
after the calculation
before the calculation
after the calculation
before the calculation
after the calculation


30

#### modifying behavior

In [58]:
def slow_calc(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, 'before_cal'):
            cb.befor_cal(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb, 'after_cal'):
            if cb.after_cal(i, val=res):
                print('stopping early!')
                break
    return res

In [59]:
class PrintAfterCallback:
    def after_cal(self, epoch, val):
        print(f'after epoch {epoch} val {val}')
        if val > 10:
            return True 

In [60]:
slow_calc(PrintAfterCallback())

after epoch 0 val 0
after epoch 1 val 1
after epoch 2 val 5
after epoch 3 val 14
stopping early!


14

In [63]:
class SlowCal:
    def __init__(self, cb=None):
        self.cb = cb
        self.res = 0
    
    def callback(self, cb_name, *args):
        if not self.cb:
            return
        cb = getattr(self.cb, cb_name, None)
        if cb:
            return cb(self, *args)
    
    def calc(self):
        for i in range(5):
            self.callback('before_cal', i)
            self.res += i*i
            sleep(1)
            if self.callback('after_cal', i):
                print('stopping early!')
                break

In [72]:
class ModifyCal:
    def after_cal(self, calc, epoch):
        print(f'After {epoch} {calc.res}')
        if calc.res > 10:
            return True
        if calc.res < 3:
            calc.res = calc.res*2

In [73]:
calc = SlowCal(ModifyCal())

In [74]:
calc.calc()

After 0 0
After 1 1
After 2 6
After 3 15
stopping early!


### __dunder__ thingies

In [84]:
class SloppyAdder:
    def __init__(self, o):
        self.o = o
    
    def __add__(self, b):
        return SloppyAdder(self.o + b.o + 0.1)
    
    def __repr__(self):
        return str(self.o)

In [85]:
a = SloppyAdder(1)
b = SloppyAdder(2)

In [86]:
a + b

3.1

In [100]:
class B:
    a,b=1,2
    
    def __getattr__(self, k):
#         print(k)
        return 'heeee'
        pass

In [101]:
c = B()

In [102]:
c.a

1

In [103]:
c.b

2

In [104]:
c.e

'heeee'